In [1]:
import time
import urllib2
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from IPython.display import IFrame
import matplotlib.font_manager as fm

plt.style.use('ggplot')
%matplotlib inline
pd.options.display.max_columns=25

In [2]:
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from bs4 import BeautifulSoup

def parse_sephora_results_page(urlbase,page):
    if page==1:
        url = urlbase
    else:
        url = urlbase + '&currentPage=' + str(page)
    print 'Parsing' + url
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"lxml")
    data = soup.find('script',attrs={'id':"searchResult"}).get_text()
    jsondata = json.loads(data)
    df=pd.DataFrame(jsondata['products']['products'])
    df['date_scraped']=datetime.now()
    return df

urlroot = 'http://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1'
#todo: get page numbers from search results
for p in np.arange(1,5):
    page_df = parse_sephora_results_page(urlroot,p)
    if p==1:
        page_df['rank']=page_df.index+1
        seph_df = page_df
    else:
        page_df['rank']=page_df.index+seph_df.shape[0]+1
        seph_df = pd.concat([seph_df,page_df])

seph_df['pct_rank']=seph_df['rank'].div(seph_df.shape[0]).mul(100)
#Reorder columns
seph_df = seph_df[['date_scraped','rank','pct_rank','brand_name','display_name','rating','id','product_url']]

filename_out = 'sephora_hair_search_' + str(datetime.now().date()) + '.xlsx'
seph_df.to_excel(filename_out,index=False)
filename_out2 = 'briogeo_sephora_hair_search_' + str(datetime.now().date()) + '.xlsx'
seph_df[seph_df['brand_name']=='Briogeo'].to_excel(filename_out2,index=False)

Parsinghttp://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1
Parsinghttp://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1&currentPage=2
Parsinghttp://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1&currentPage=3
Parsinghttp://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1&currentPage=4


In [8]:
print response.content
soup = BeautifulSoup(response.content,"lxml")


<!DOCTYPE html>
<html lang="en" class="no-js">
<head>

  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <title>Sephora</title>
	<meta name="apple-mobile-web-app-capable" content="yes">
  <meta name="apple-mobile-web-app-status-bar-style" content="black">
  <meta name="apple-mobile-web-app-title" content="Sephora">
  <meta name="viewport" content="width=1024">
  <meta name="SKYPE_TOOLBAR" content="SKYPE_TOOLBAR_PARSER_COMPATIBLE">
  <meta name="format-detection" content="telephone=no">
  <meta name="contextPath" content="">
  <meta name="imageBasePath" content="/images/">
  <meta name="bvctx" content="">
  <meta name="google-site-verification" content="DHEfxRvk5978fvUKaGrOCpRByg5EXyhJBxhEbM0rZKk" />

  <meta name="is_olr_myreservations_enabled" content="true"/>
  <meta name="sitescope" content="ok"/>

    <script>
	(function(){
	  var dom,doc,where,iframe = document.createElement('iframe');
	  iframe.src = "javascript:false";
	  (iframe.frameElement || 

In [34]:
data = soup.find('script',attrs={'id':"searchResult"}).get_text()
output = json.loads(data)
#print output
print output['products'].viewkeys()
#pd.read_json(output['products']['products'],orient='records')
df=pd.DataFrame(output['products']['products'])

dict_keys([u'refinements', u'total_products', u'products', u'categories', u'page_size'])


In [37]:
df['rank']=df.index+1
df['date_scraped']=datetime.now()
df = df[['date_scraped','rank','brand_name','display_name','rating','id','product_url']]
df.head()

,date_scraped,rank,brand_name,display_name,rating,id,product_url
0,2017-01-14 17:07:20.100905,1,dyson,Supersonic Hair Dryer,4.3881,P410214,/supersonic-hair-dryer-P410214
1,2017-01-14 17:07:20.100905,2,Josie Maran,100 percent Pure Argan Oil,4.5165,P218700,/100-percent-pure-argan-oil-P218700
2,2017-01-14 17:07:20.100905,3,T3,Whirl Trio Interchangeable Styling Wand,4.5625,P388707,/whirl-trio-interchangeable-styling-wand-P388707
3,2017-01-14 17:07:20.100905,4,Living Proof,Perfect Hair Day Dry Shampoo,4.1872,P399330,/perfect-hair-day-dry-shampoo-P399330
4,2017-01-14 17:07:20.100905,5,Drybar,Let it Blow! It’s Drybar to Go The Ultimate Tr...,4.2581,P411590,/let-it-blow-it-s-drybar-to-go-ultimate-travel...


In [39]:
df['brand_name'].unique()

array([u'dyson', u'Josie Maran', u'T3', u'Living Proof', u'Drybar',
       u'Bio Ionic', u'Keranique', u'Moroccanoil', u'Bumble and bumble',
       u'ALTERNA Haircare', u'Madison Reed', u'Ouai', u'amika', u'Klorane',
       u'ghd', u'Tangle Teezer', u'WEN\xae by Chaz Dean',
       u'SEPHORA COLLECTION', u'Phyto', u'Christophe Robin',
       u'Shhhowercap', u'Rita Hazan', u'invisibobble', u'DevaCurl',
       u'Briogeo', u'Jack Black', u'Reverie', u'Captain Blankenship',
       u'Elizabeth and James', u'dpHUE', u'Aquis', u'Verb', u'Tatcha',
       u'Sachajuan', u'IGK', u'Rahua', u'Caudalie', u'Kopari',
       u'Madam C.J. Walker Beauty Culture', u'Iles Formula',
       u'Sephora Favorites', u'Ouidad', u'Qhemet Biologics',
       u'Sally Hershberger 24K', u'Gloss Moderne'], dtype=object)

In [40]:
df[df['brand_name']=='Briogeo']

,date_scraped,rank,brand_name,display_name,rating,id,product_url
77,2017-01-14 17:07:20.100905,78,Briogeo,"Don’t Despair, Repair!™ Deep Conditioning Mask",4.0015,P388628,/dont-despair-repair-deep-conditioning-mask-P3...
159,2017-01-14 17:07:20.100905,160,Briogeo,Blossom & Bloom™ Ginseng + Biotin Volumizing S...,4.1758,P402071,/blossom-bloom-ginseng-biotin-volumizing-shamp...
220,2017-01-14 17:07:20.100905,221,Briogeo,Blossom & Bloom™ Ginseng + Biotin Volumizing C...,4.2910,P388625,/blossom-bloom-ginseng-biotin-volumizing-condi...
238,2017-01-14 17:07:20.100905,239,Briogeo,Rosarco Milk™ Reparative Leave-In Conditioning...,4.3527,P396733,/rosarco-milk-reparative-leave-in-conditioning...


In [42]:
print df.shape
df

(300, 7)


,date_scraped,rank,brand_name,display_name,rating,id,product_url
0,2017-01-14 17:07:20.100905,1,dyson,Supersonic Hair Dryer,4.3881,P410214,/supersonic-hair-dryer-P410214
1,2017-01-14 17:07:20.100905,2,Josie Maran,100 percent Pure Argan Oil,4.5165,P218700,/100-percent-pure-argan-oil-P218700
2,2017-01-14 17:07:20.100905,3,T3,Whirl Trio Interchangeable Styling Wand,4.5625,P388707,/whirl-trio-interchangeable-styling-wand-P388707
3,2017-01-14 17:07:20.100905,4,Living Proof,Perfect Hair Day Dry Shampoo,4.1872,P399330,/perfect-hair-day-dry-shampoo-P399330
4,2017-01-14 17:07:20.100905,5,Drybar,Let it Blow! It’s Drybar to Go The Ultimate Tr...,4.2581,P411590,/let-it-blow-it-s-drybar-to-go-ultimate-travel...
5,2017-01-14 17:07:20.100905,6,Bio Ionic,"10X Pro Styling Iron 1""",4.7895,P406232,/10x-pro-styling-iron-1-P406232
6,2017-01-14 17:07:20.100905,7,Keranique,Deluxe Regrowth Treatment & Deep Hydration Kit,3.9259,P410735,/deluxe-regrowth-treatment-deep-hydration-kit-...
7,2017-01-14 17:07:20.100905,8,Moroccanoil,Moroccanoil Treatment,4.7700,P412090,/moroccanoil-treatment-P412090
8,2017-01-14 17:07:20.100905,9,Living Proof,Restore Perfecting Spray,0.0000,P415925,/restore-perfecting-spray-P415925
9,2017-01-14 17:07:20.100905,10,Bumble and bumble,Hairdresser’s Invisible Oil Primer,4.5749,P386464,/hairdresser-s-invisible-oil-primer-P386464


In [24]:
output['products'].viewkeys()

dict_keys([u'refinements', u'total_products', u'products', u'categories', u'page_size'])

In [6]:
import json, urllib
data = urllib.urlopen(url).read()
output = json.loads(data)
print (output)

ValueError: No JSON object could be decoded

In [3]:
urlroot = 'http://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1'
pnum=1
productlinklist = [] 
url = urlroot+'?currentPage='+str(pnum)
print 'Parsing '+url

Parsing http://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1?currentPage=1


In [7]:
resp2 = requests.get(urlroot)

NameError: name 'urlroot' is not defined

In [ ]:
import mechanize
br = mechanize.Browser()
import cookielib
cj = cookielib.LWPCookieJar()
br.set_cookiejar(cj)
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
data = br.open(resp2.url).get_data()

In [ ]:
print data

In [10]:
soup = BeautifulSoup(data,"lxml")
#soup.
'''
<a ng-href="/supersonic-hair-dryer-P410214?skuId=1870179&amp;icid2=products grid:p410214" class="u-size1of4 SkuItem SkuItem--135 SkuItem SkuItem--135" data-at="sku_item_1870179" seph-sku-item="" ana-click-thru="" ng-repeat="sku in skugroup.sku_list" href="/supersonic-hair-dryer-P410214?skuId=1870179&amp;icid2=products grid:p410214">
  <div>
    <div class="has-ql u-mxa" ng-style="::{'max-width': imageSize}" style="max-width: 135px;">
      <div class="FlexEmbed">
        <div class="FlexEmbed-ratio"></div>
        <img class="FlexEmbed-content" data-lazyload="" seph-lazy-src="/productimages/sku/s1870179-main-grid.jpg" src="/productimages/sku/s1870179-main-grid.jpg">
      </div>
      <!-- ngIf: ::sku.is_beauty_insider -->
      <!-- ngIf: ::!skugroup.hideQuickLook --><span ng-if="::!skugroup.hideQuickLook" seph-quick-look="" data-product-id="P410214" data-sku_number="1870179" class="ShowQL ng-scope" data-at="ql_show" ana-event-info="{&quot;clickOrigin&quot;:{&quot;componentSubType&quot;: &quot;&quot;}}">Quick Look</span><!-- end ngIf: ::!skugroup.hideQuickLook -->
    </div>
  </div>
  <div class="SkuItem-newLove">
    <!-- ngIf: ::sku.is_new -->
    <!-- ngIf: ::(skugroup.is_loves_shown && sku.isLoveEligible()) --><div class="SkuItem-love ng-scope" ng-if="::(skugroup.is_loves_shown &amp;&amp; sku.isLoveEligible())">
      <button type="button" seph-tooltip="" uib-tooltip-html="loveTooltip" tooltip-append-to-body="true" class="LoveIcon is-hoverable" seph-lovable="P410214" ng-class="{'is-loved': isLoved, 'is-hoverable': hoverEnable}" data-sku_number="1870179" data-at="sku_item_love"><svg class="Icon Icon--love"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-love"></use></svg>
<svg class="Icon Icon--loveOutline"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-loveOutline"></use></svg>
<svg class="Icon Icon--loveHover"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-loveHover"></use></svg>
</button>
    </div><!-- end ngIf: ::(skugroup.is_loves_shown && sku.isLoveEligible()) -->
  </div>
  <div class="SkuItem-info">
    <!-- ngIf: ::skugroup.is_numbers_shown -->
    <div class="SkuItem-name">
      <div class="SkuItem-nameBrand OneLinkNoTx ng-binding" data-at="sku_item_brand" ng-bind-html="trust(sku.primary_product.brand_name)">dyson</div>
      <div class="SkuItem-nameDisplay ng-binding" data-at="sku_item_name" ng-bind-html="trust(sku.primary_product.display_name)">Supersonic Hair Dryer</div>
    </div>
    <!-- ngIf: ::skugroup.is_price_shown!==false --><div class="u-fwb ng-scope" ng-if="::skugroup.is_price_shown!==false">
      <span data-at="sku_item_price_list" ng-class="::{'u-fwn u-strike u-midGray': sku.sale_price || sku.sale_price_min}" class="ng-binding">$399.00</span>
      <!-- ngIf: ::sku.sale_price -->
      <!-- ngIf: ::sku.sale_price_min -->
      <!-- ngIf: ::sku.value_price -->
    </div><!-- end ngIf: ::skugroup.is_price_shown!==false -->
    <!-- ngIf: ::skugroup.flags --><div class="SkuItem-flags u-mt1 u-mb1 ng-scope" data-at="sku_item_flags" ng-if="::skugroup.flags">
      <!-- ngIf: ::sku.is_sephora_exclusive -->
      <!-- ngIf: ::sku.is_limited_edition -->
      <!-- ngIf: ::sku.is_online_only -->
    </div><!-- end ngIf: ::skugroup.flags -->
    <!-- ngIf: ::(sku.primary_product.more_colors && !sku.primary_product.isNoShowMore()) --><div ng-if="::(sku.primary_product.more_colors &amp;&amp; !sku.primary_product.isNoShowMore())" class="SkuItem-colors u-mt2 u-mb2 ng-binding ng-scope">
      [ 1 more <span ng-pluralize="" count="::sku.primary_product.more_colors" when="{'one': 'color', 'other': 'colors'}">color</span> ]
    </div><!-- end ngIf: ::(sku.primary_product.more_colors && !sku.primary_product.isNoShowMore()) -->
    <!-- ngIf: ::(skugroup.is_rating_shown && !sku.primary_product.is_hide_social) --><div class="u-db u-mxa u-mt2 u-mb2 StarRating u-relative u-oh ng-scope" ng-if="::(skugroup.is_rating_shown &amp;&amp; !sku.primary_product.is_hide_social)" seph-stars="4.3735">
  <div class="StarRating-set u-moonGray">
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
  </div>
  <div class="StarRating-set u-absolute u-top0 u-left0 u-oh u-red" ng-style="{'width': rating+'%'}">
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
    <svg class="Icon Icon--star"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="/images/icon.svg#icon-star"></use></svg>
  </div>
</div><!-- end ngIf: ::(skugroup.is_rating_shown && !sku.primary_product.is_hide_social) -->
  </div>
  <!-- ngIf: ::( (skugroup.is_use_add_to_basket||sku.has_add_btn||abTests.addBtnBasket) && !sku.isOutOfStock() && !sku.noShipToCAorUS) -->
  <!-- ngIf: ::(sku.isOutOfStock() && !sku.noShipToCAorUS && skugroup.is_show_sign_up_for_email) -->
  <!-- ngIf: ::sku.noShipToCAorUS -->
</a>
'''

#productlist = soup.find('div',attrs={'class':"SkuGrid"})
#productlist = soup.find('a',attrs={'class':"u-size1of4.SkuItem.SkuItem--135"})
#productlist = soup.find('a',attrs={'class':"SkuItem--135"})
productlist = soup.find_all('div',attrs={'class':"SkuItem--135"})
#productlist = soup.find()
print len(productlist)
print productlist
#print productlist.findChildren()
#for child in children:
#    print child

0
[]


In [3]:
response.

TypeError: 'unicode' object is not callable

In [ ]:
def check_party_date(pdat,datecheck):
    #Extract the span that contains the date
    party_date = pdat.find('span',attrs={'class': 'views-field views-field-created'}).text.strip()
    return datetime.strptime(party_date,'%A, %B %d, %Y') < datecheck

def parse_product_lists(urlroot,pnum):
    productlinklist = [] 
    url = urlroot+'?currentPage='+str(pnum)
    print 'Parsing '+url
    #Grab the whole page
    #To handle the occasional 503 error, use a try statement and wait 5 seconds if HTTPError
    try:
        raw_page = urllib2.urlopen(url).read()
    except urllib2.HTTPError as e:
        print e.code
        time.sleep(5)
        raw_page = urllib2.urlopen(url).read()
    #Process the page w/ BeautifulSoup
    soup = BeautifulSoup(raw_page)
    partylist = soup.find('div',attrs={'class': 'view-content'})
    parties = partylist.find_all('div',attrs={'class': 'views-row'})
    for p in parties:
        # check the date to ensure it is before Dec 1, 2014
        if check_party_date(p,datetime(2014, 12, 1, 0,0)):
        # use the href link to get our link to the party
          party_link = urlroot+p.find('a').attrs['href']
        # skip the following odd link if it comes up
          if party_link != 'http://www.newyorksocialdiary.com/nysd/partypictures':
            # add to party link list
              partylinklist.append(party_link)
    return partylinklist

urlroot = 'http://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1'
for page in np.arange(1,4):
    products = parse_product_lists(urlroot,page)
print parties[:15]

In [4]:
raw_page = urllib2.urlopen(url).read()
print raw_page

HTTPError: HTTP Error 403: Forbidden

In [27]:
parties = partylist[0].find_all('a',attrs={'class': 'SkuItem--135'})
for p in parties:
    print p

KeyError: 0

In [26]:
print len(partylist)

3


In [49]:
print datetime.now().date()

2017-01-14


In [5]:
filelist=['files/sephora_hair_search_2017-01-18.xlsx',
'files/sephora_hair_search_2017-01-19.xlsx',
'files/sephora_hair_search_2017-01-23.xlsx',
'files/sephora_hair_search_2017-01-26.xlsx',
'files/sephora_hair_search_2017-01-28.xlsx',
'files/sephora_hair_search_2017-01-29.xlsx',
'files/sephora_hair_search_2017-02-01.xlsx',
'files/sephora_hair_search_2017-02-02.xlsx',
'files/sephora_hair_search_2017-02-05.xlsx',
'files/sephora_hair_search_2017-02-07.xlsx',
'files/sephora_hair_search_2017-02-09.xlsx']
all_df = pd.read_excel('files/sephora_hair_search_2017-01-14.xlsx')
for f in filelist:
    this_df=pd.read_excel(f)
    all_df=all_df.append(this_df,ignore_index=True)
print all_df.shape

(12416, 8)


In [6]:
all_df.to_excel('files/sephora_hair_concat.xlsx',index=False)

In [2]:
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from bs4 import BeautifulSoup

urlroot = 'http://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1'
page=1
url=urlroot
print 'Parsing' + url
response = requests.get(url)
soup = BeautifulSoup(response.content,"lxml")
data = soup.find('script',attrs={'id':"searchResult"}).get_text()
jsondata = json.loads(data)
    

Parsinghttp://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1


In [14]:
with open('jsondata.txt', 'w') as outfile:
    json.dump(data, outfile)

In [7]:
print jsondata.keys()

[u'brand', u'deals', u'photos', u'meta', u'products', u'videos_ooyala', u'warning_message']


In [6]:
print jsondata['products'].keys()

[u'refinements', u'total_products', u'products', u'categories', u'page_size']


In [8]:
print jsondata['meta'].keys()

[u'locale', u'tabs', u'image_path', u'currency_symbol']


In [ ]:
    
df=pd.DataFrame(jsondata['products']['products'])
df['date_scraped']=datetime.now()
return df

urlroot = 'http://www.sephora.com/search/search.jsp?keyword=hair&mode=all&node=1050092&sortBy=P_BEST_SELLING%3A1%3A%3AP_RATING%3A1%3A%3AP_PROD_NAME%3A0%3A%3AP_DEFAULT_SKU%3A1&pageSize=-1'
page=1
url=urlroot
    page_df = parse_sephora_results_page(urlroot,p)
    if p==1:
        page_df['rank']=page_df.index+1
        seph_df = page_df
    else:
        page_df['rank']=page_df.index+seph_df.shape[0]+1
        seph_df = pd.concat([seph_df,page_df])

seph_df['pct_rank']=seph_df['rank'].div(seph_df.shape[0]).mul(100)
#Reorder columns
seph_df = seph_df[['date_scraped','rank','pct_rank','brand_name','display_name','rating','id','product_url']]

filename_out = 'sephora_hair_search_' + str(datetime.now().date()) + '.xlsx'
seph_df.to_excel(filename_out,index=False)
filename_out2 = 'briogeo_sephora_hair_search_' + str(datetime.now().date()) + '.xlsx'
seph_df[seph_df['brand_name']=='Briogeo'].to_excel(filename_out2,index=False)

In [2]:
urlroot = 'http://www.sephora.com'
page1='/rosarco-oil-P388629'
page2='/blossom-bloom-ginseng-biotin-volumizing-shampoo-P402071'
url = urlroot + str(page1)
print 'Parsing' + url
response = requests.get(url)
soup = BeautifulSoup(response.content,"lxml")

Parsinghttp://www.sephora.com/rosarco-oil-P388629


In [4]:
print soup.find('b',attrs={'class','u-textWarning'})

<b class="u-textWarning">
<span ng-show="sku.is_coming_soon">Coming soon</span>
<span ng-hide="sku.is_coming_soon">Out of stock</span>
</b>


In [9]:
print soup.find('script')

TypeError: 'NoneType' object is not callable

In [10]:
soup.find_all('script')

[<script>if (typeof global === "undefined") window.global = window;global.Sephora = global.Sephora || {};Sephora.userFullQueryAppend = "";</script>,
 <script>"use strict";!function(){Sephora.isRootRender=!1,Sephora.debug={},Sephora.Util={},Sephora.Certona={},Sephora.Util.InflatorComps={},Sephora.analytics={backendData:{}},Sephora.isMobile=function(){return"MW"===Sephora.channel},Sephora.isDesktop=function(){return"FS"===Sephora.channel},Sephora.isTouch="ontouchstart"in window||window.DocumentTouch&&document instanceof DocumentTouch,global.digitalData={};var e=Sephora.Util.InflatorComps.services={};e.loadEvents={},Sephora.loadFunctions={onLoad:function(){Sephora.LoadFired=!0,e.loadEvents.load=!0,console.log(performance.now()+": Load Event Fired")},onDomContentLoad:function(){Sephora.DOMContentLoadedFired=!0,e.loadEvents.DOMContentLoaded=!0,console.log(performance.now()+": DOMContentLoaded")}},"complete"===document.readyState||"loaded"===document.readyState?Sephora.loadFunctions.onDomCon

In [5]:
urlroot = 'http://www.sephora.com'
page1='/rosarco-oil-P388629'
page2='/blossom-bloom-ginseng-biotin-volumizing-shampoo-P402071'
url2 = urlroot + str(page2)
print 'Parsing ' + url2
response2 = requests.get(url2)
soup2 = BeautifulSoup(response.content,"lxml")

Parsinghttp://www.sephora.com/blossom-bloom-ginseng-biotin-volumizing-shampoo-P402071


In [6]:
print soup.find('b',attrs={'class','u-textWarning'})

<b class="u-textWarning">
<span ng-show="sku.is_coming_soon">Coming soon</span>
<span ng-hide="sku.is_coming_soon">Out of stock</span>
</b>


In [ ]:
data = soup.find('script',attrs={'id':"searchResult"}).get_text()
jsondata = json.loads(data)
df=pd.DataFrame(jsondata['products']['products'])

In [3]:
oos_item='https://www.sephora.com/product/dont-despair-repair-deep-conditioning-mask-P388628?skuId=1784636'
is_item='https://www.sephora.com/product/dont-despair-repair-deep-conditioning-mask-P388628?skuId=1823418'
resp_oos=requests.get(oos_item)
resp_is=requests.get(is_item)

In [13]:
import re
print re.search('Stock',resp_oos.content)
print re.search('Stock',resp_is.content)

In [15]:
with open("oos.txt", "w") as text_file:
    text_file.write("{}".format(resp_oos.content))
    
with open("is.txt", "w") as text_file:
    text_file.write("{}".format(resp_is.content))


In [16]:
soup = BeautifulSoup(resp_oos.content,"lxml")
data = soup.find_all('button')
    

In [17]:
print soup.find('script',attrs={'id':"searchResult"}).get_text()

[<button class="css-1qp46xb" data-comp="ButtonPrimary Button Base" data-reactid="19" type="button">Sign In</button>, <button class="css-1pbtupk" data-comp="Link Base" data-reactid="24"><!-- react-text: 25 -->basket<!-- /react-text --></button>, <button class="css-fisw11" data-comp="RatingsSummary Box Base" data-reactid="87"><div class="css-1m1iwm2" data-comp="StarRating Box Base" data-reactid="88"><div class="css-zpsycc" data-comp="Box Base" data-reactid="89"><svg class="css-89svy6" data-comp="IconStar Icon Base" data-reactid="90" viewbox="0 0 32 32"><path d="M16 0l4.9 10.5L32 12.2l-8 8.2L25.9 32 16 26.5 6.1 32 8 20.4l-8-8.2 11.1-1.7L16 0z" data-reactid="91"></path></svg><svg class="css-89svy6" data-comp="IconStar Icon Base" data-reactid="92" viewbox="0 0 32 32"><path d="M16 0l4.9 10.5L32 12.2l-8 8.2L25.9 32 16 26.5 6.1 32 8 20.4l-8-8.2 11.1-1.7L16 0z" data-reactid="93"></path></svg><svg class="css-89svy6" data-comp="IconStar Icon Base" data-reactid="94" viewbox="0 0 32 32"><path d="M1

In [34]:
for link in soup.find_all('script'):
    if "Sephora.Util.InflatorComps.queue('RegularProductTop'," in link.get_text():
        #print link.get_text()[55:-16]
        json_string=link.get_text()[55:-16].replace('\\"','"').replace("\\'","'").replace('\\"','"')
        print json_string
        jsondata = json.loads(json_string)
        #print jsondata
        #df=pd.DataFrame(jsondata['products']['products'])

{"user":{},"currentProduct":{"seoProductName":"dont-despair-repair-deep-conditioning-mask-P388628","seoName":"dont-despair-repair-deep-conditioning-mask-P388628","seoTitle":"Don’t Despair, Repair!™ Deep Conditioning Mask - Briogeo | Sephora","seoMetaDescription":"Shop Briogeo’s Don't Despair, Repair!™ Deep Conditioning Mask at Sephora. This mask transforms dull, dry, and lackluster locks. ","seoKeywords":[],"targetUrl":"\/product\/dont-despair-repair-deep-conditioning-mask-P388628","seoCanonicalUrl":"\/product\/dont-despair-repair-deep-conditioning-mask-P388628","productId":"P388628","displayName":"Don’t Despair, Repair!™ Deep Conditioning Mask","variationType":"Size","type":"standard","skuSelectorType":"Text","swatchType":"Image - 36","imageAltText":"Briogeo - Don’t Despair, Repair!™ Deep Conditioning Mask","regularChildSkus":[{"targetUrl":"\/product\/dont-despair-repair-deep-conditioning-mask-P388628","skuId":"1823418","displayName":"1823418 1 oz\/ 30 mL","listPrice":"$9.00","isBiOnl

In [33]:
print json_string[15690:15700]

 \\"full o


In [48]:
print jsondata.keys()

[u'currentProduct', u'user']


In [38]:
print len(jsondata['currentProduct'])
print jsondata['currentProduct'].keys()

55
[u'suggestedUsage', u'rating', u'siteLocale', u'quickLookDescription', u'ymalProductTitle', u'ajaxTimeout', u'imageAltText', u'seoCanonicalUrl', u'breadcrumbsSeoJsonLd', u'shortDescription', u'skuSelectorType', u'apiConfigurationData', u'productArticles', u'thirdpartyImageHost', u'fullSiteProductUrl', u'seoName', u'ajaxMaxRetries', u'content', u'variationType', u'biOptions', u'targetUrl', u'ufeSeoCanonicalUrl', u'type', u'productId', u'seoKeywords', u'isProduction', u'useThirdPartyProduction', u'parentCategory', u'brightTagId', u'brand', u'navigationSeoJsonLd', u'longDescription', u'seoTitle', u'ajaxExtendedTimeout', u'currentSku', u'ajaxExtendedTimeoutFindInStore', u'regularChildSkus', u'productSeoJsonLd', u'seoMetaDescription', u'simillarSkus', u'displayName', u'isHideSocial', u'templateInformation', u'headerFooterTemplate', u'productVideos', u'reviews', u'enableScan', u'useSiteCatalystOutsideSignal', u'ymalSkus', u'isMboxEnabled', u'ufeSeoAlternatelUrl', u'fullSiteHostName', u'lo

In [49]:
print jsondata['currentProduct']['imageAltText']

Briogeo - Don’t Despair, Repair!™ Deep Conditioning Mask


In [42]:
print len(jsondata['currentProduct']['regularChildSkus'])

2


In [46]:
print jsondata['currentProduct']['regularChildSkus'][0]

{u'isFindInStore': False, u'isNew': False, u'isPaypalRestricted': False, u'size': u'1 oz/ 30 mL', u'ingredientDesc': u' -Rosehip Oil: Maintains healthy hair hydration and controls frizz with rich, essential fatty acids. \\r\\n<br>-Algae Extract: Nourishes and strengthens the hair and protects the scalp from free-radical damage with a host of minerals, amino acids, antioxidants, and vitamins. \\r\\n<br>-B Vitamins: Improve the texture of the hair and deliver body and sheen, while working to repair split ends and restore nourishment to reverse heat damage or chemically-treated locks. \\r\\n<br>-Almond Oil: Enhances shine and luster of dry or weak hair and smooths the cuticle for moisture-locking, anti-frizz benefits. \\r\\n<br><br>\\r\\nWater, Stearyl Alcohol*, Cetyl Alcohol*, Brassicyl Isoleucinate Esylate*, Propanediol*, Brassica Alcohol*, Isopropyl Myristate*, Panthenol, Biotin, Argania Spinosa Kernel Oil, Persea Gratissima (Avocado) Oil, Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ro

In [45]:
print jsondata['currentProduct']['regularChildSkus'][0]['isOutOfStock']

False


In [47]:
for sku in jsondata['currentProduct']['regularChildSkus']:
    print sku['targetUrl'], sku['skuName'], sku['isOutOfStock']
    if sku['isOutOfStock']:
        with open("oos_alert_email.txt", "w") as text_file:
            text_file.write("{0}".format())

/product/dont-despair-repair-deep-conditioning-mask-P388628 1 oz/ 30 mL False
/product/dont-despair-repair-deep-conditioning-mask-P388628?skuId=1784636 8 oz/ 236 mL True


In [80]:
urlroot='http://sephora.com'
prod_urls=['/product/dont-despair-repair-deep-conditioning-mask-P388628',
'/product/scalp-revival-charcoal-coconut-oil-micro-exfoliating-shampoo-P418507',
'/product/rosarco-milk-reparative-leave-in-conditioning-spray-P396733',
'/product/scalp-revival-charcoal-tea-tree-scalp-treatment-P418506',
'/product/rosarco-blow-dry-perfection-heat-protectant-creme-P411359',
'/product/blossom-bloom-ginseng-biotin-volumizing-conditioner-P388625',
'/product/blossom-bloom-ginseng-biotin-volumizing-shampoo-P402071',
'/product/scalp-revival-charcoal-biotin-dry-shampoo-P418505',
'/product/be-gentle-be-kind-avocado-quinoa-co-wash-P388623',
'/product/curl-charisma-rice-amino-avocado-leave-in-defining-creme-P388626',
'/product/rosarco-repair-conditioner-P402073',
'/product/be-gentle-be-kind-green-tea-clarifying-shampoo-P388624',
'/product/curl-charisma-definition-on-the-go-travel-kit-P422368',
'/product/rosarco-repair-shampoo-P402072',
'/product/rosarco-repair-on-the-go-travel-kit-P422369',
'/product/curl-charisma-rice-amino-quinoa-frizz-control-gel-P408411',
'/product/curl-charisma-rice-amino-shampoo-P402074',
'/product/curl-charisma-rice-amino-shea-curl-defining-conditioner-P388627',
'/product/blossom-bloom-volumizing-on-the-go-travel-kit-P422370',
'/product/rosarco-oil-P388629',
'/product/don-t-despair-repair-gel-to-oil-overnight-repair-treatment-P408248',
'/product/blossom-bloom-ginseng-biotin-volumizing-spray-P396734',
'/product/ultimate-hair-goals-best-briogeo-kit-P425022',
'/product/rosarco-repair-winter-hair-renewal-P425021']
oos_flag=False
email_filename = 'oos_email_' + str(datetime.now().date()) + '.txt'
import os.path

if ~os.path.isfile(email_filename):

    text_file = open(email_filename, "w")

    for p in prod_urls:
        print p
        resp=requests.get(urlroot+p)
        soup = BeautifulSoup(resp.content,"lxml")
        for link in soup.find_all('script'):
            if "Sephora.Util.InflatorComps.queue('RegularProductTop'," in link.get_text():
                #print link.get_text()[55:-16]
                json_string=link.get_text()[55:-16].replace('\\"','"').replace("\\'","'").replace('\\"','"')
                #print json_string
                jsondata = json.loads(json_string)
                if 'regularChildSkus' in jsondata['currentProduct'].keys():
                    field='regularChildSkus'
                    for sku in jsondata['currentProduct'][field]:
                        print sku['targetUrl'], sku['skuName'], sku['isOutOfStock']
                        if sku['isOutOfStock']:
                            oos_flag=True
                            text_file.write("\nOOS: {0} {1} {2}\n".format(urlroot+sku['targetUrl'], sku['skuName'], sku['isOutOfStock']))
                else:
                    field='currentSku'
                    sku=jsondata['currentProduct'][field]
                    print sku['targetUrl'], sku['isOutOfStock']
                    if sku['isOutOfStock']:
                        oos_flag=True
                        text_file.write("\nOOS: {0} {1}\n".format(urlroot+sku['targetUrl'], sku['isOutOfStock']))


    text_file.close()                
    if oos_flag:
        email_nancy(email_filename)

 /product/dont-despair-repair-deep-conditioning-mask-P388628
/product/dont-despair-repair-deep-conditioning-mask-P388628 1 oz/ 30 mL False
/product/dont-despair-repair-deep-conditioning-mask-P388628?skuId=1784636 8 oz/ 236 mL True
/product/scalp-revival-charcoal-coconut-oil-micro-exfoliating-shampoo-P418507
/product/scalp-revival-charcoal-coconut-oil-micro-exfoliating-shampoo-P418507 False
/product/rosarco-milk-reparative-leave-in-conditioning-spray-P396733
/product/rosarco-milk-reparative-leave-in-conditioning-spray-P396733 5 oz False
/product/rosarco-milk-reparative-leave-in-conditioning-spray-P396733?skuId=1823426 1.75 oz False
/product/scalp-revival-charcoal-tea-tree-scalp-treatment-P418506
/product/scalp-revival-charcoal-tea-tree-scalp-treatment-P418506 False
/product/rosarco-blow-dry-perfection-heat-protectant-creme-P411359
/product/rosarco-blow-dry-perfection-heat-protectant-creme-P411359 False
/product/blossom-bloom-ginseng-biotin-volumizing-conditioner-P388625
/product/blossom

error: [Errno 61] Connection refused

In [78]:
#print jsondata['currentProduct'][field]
#print jsondata['currentProduct'].keys()
#print jsondata['currentProduct']['currentSku']
with open("curltravel.txt", "w") as text_file:
    text_file.write("{}".format(resp.content))

In [58]:
print "OOS: {0} {1} {2}".format(urlroot+sku['targetUrl'], sku['skuName'], sku['isOutOfStock'])
text_file = open(email_filename, "w")
text_file.write("\nOOS: {0} {1} {2}\n".format(urlroot+sku['targetUrl'], sku['skuName'], sku['isOutOfStock']))
text_file.close()

OOS: http://sephora.com/product/dont-despair-repair-deep-conditioning-mask-P388628?skuId=1784636 8 oz/ 236 mL True


In [90]:
import smtplib

# Import the email modules we'll need
from email.mime.text import MIMEText

def email_nancy(text_file):
    # Open a plain text file for reading.  For this example, assume that
    # the text file contains only ASCII characters.
    fp = open(text_file, 'rb')
    # Create a text/plain message
    msg = MIMEText(fp.read())
    fp.close()

    me = 'btquinn@gmail.com'
    you ='orders@briogeohair.com'
    msg['Subject'] = 'SEPHORA OUT OF STOCK ALERT'
    msg['From'] = me
    msg['To'] = you

    # Send the message via our own SMTP server, but don't include the
    # envelope header.
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    #Next, log in to the server
    server.login("btquinn", "ilC.20090521")

    server.sendmail(me, you, msg.as_string())
    server.quit()


In [91]:
email_nancy(email_filename)

In [82]:
import smtplib
server = smtplib.SMTP('smtp.gmail.com', 587)

#Next, log in to the server
server.login("btquinn", "ilC.20090521")

#Send the mail
msg = '''
Hello!''' # The /n separates the message from the headers
server.sendmail("btquinn@gmail.com", "nancy@briogeohair.com", msg)

SMTPException: SMTP AUTH extension not supported by server.

In [83]:
server.ehlo()
server.starttls()
server.ehlo()

(250,
 'smtp.gmail.com at your service, [65.209.60.146]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [85]:
server.login("btquinn", "ilC.20090521")


(235, '2.7.0 Accepted')

In [86]:
msg = '''
Hello!''' # The /n separates the message from the headers
server.sendmail("btquinn@gmail.com", "nancy@briogeohair.com", msg)

{}

In [99]:
import os
if not os.path.isfile(email_filename):
    print 'no file'